In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import numpy as np
from PIL import Image
import glob

images = []
for file in glob.glob('../../IITD Palmprint V1/Segmented/Left/*.bmp'): # Put the path to the dataset here
    image = Image.open(file)
    numpy_array = np.array(image)
    images.append(numpy_array)
for file in glob.glob('../../IITD Palmprint V1/Segmented/Right/*.bmp'): # Put the path to the dataset here
    image = Image.open(file)
    numpy_array = np.array(image)
    images.append(numpy_array)

palm_arrays = np.array(images)

In [3]:
x_train = palm_arrays[:2000]
x_test = palm_arrays[2000:]
print(x_train.shape)
print(x_test.shape)

In [4]:
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_train = x_train.reshape(-1, 150, 150, 1)
x_test = x_test.reshape(-1, 150, 150, 1)
print(x_train.shape)
print(x_test.shape)

In [5]:
import tensorflow as tf
from tensorflow import keras

input_img = keras.Input(shape=(150,150,1)) # shape = (width, height, channels)
encoded = keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(input_img)
encoded = keras.layers.MaxPooling2D((2,2))(encoded)
encoded = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(encoded)
encoded = keras.layers.MaxPooling2D((3,3))(encoded)
# encoded = keras.layers.Conv2D(1, (3,3), activation='relu', padding='same')(encoded)
encoded = keras.layers.Flatten()(encoded)
encoded = keras.layers.Dropout(0.3)(encoded)
# encoded = keras.layers.Dense(24*24, activation='relu')(encoded)
encoded = keras.layers.Dense(256, activation='relu')(encoded)
encoded = keras.layers.Dense(64, activation='relu')(encoded)
# encoded = keras.layers.Dense(32, activation='relu')(encoded)

# decoded = keras.layers.Dense(32, activation='relu')(encoded)
decoded_dense1 = keras.layers.Dense(64, activation='relu')(encoded)
decoded_dense2 = keras.layers.Dense(256, activation='relu')(decoded_dense1)
decoded_dense3 = keras.layers.Dense(25*25, activation='relu')(decoded_dense2)
decoded_reshaped = keras.layers.Reshape((25,25,1))(decoded_dense3)
decoded_conv0 = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(decoded_reshaped)
decoded_upSample0 = keras.layers.UpSampling2D((3,3))(decoded_conv0)
decoded_conv1 = keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(decoded_upSample0)
decoded_upSample1 = keras.layers.UpSampling2D((2,2))(decoded_conv1)
decoded = keras.layers.Conv2D(1, (3,3), activation='relu', padding='same')(decoded_upSample1)


autoencoder = keras.Model(input_img, decoded)

In [6]:
# Encoder Model
encoder = keras.Model(input_img, encoded)

In [7]:
# encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [8]:
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.mae)

In [9]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 74, 74, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 512)       590336

In [10]:
autoencoder.fit(x_train, x_train, epochs=2, shuffle=True)

In [11]:
encoded_palms = encoder.predict(x_test)

In [12]:
encoded_palms[1]

array([   0.      , 2185.516   , 2854.9968  ,  434.45908 , 1940.6245  ,
       2150.4453  ,    0.      , 2255.9976  ,  666.42694 ,    0.      ,
          0.      , 1380.3615  , 1550.857   ,  436.58313 ,  563.3056  ,
        899.8952  ,    0.      ,    0.      , 2885.544   ,  286.46417 ,
         21.755493,    0.      ,    0.      ,    0.      ,  493.04102 ,
        941.0677  , 2786.606   , 2156.8945  ,    0.      ,    0.      ,
        346.30478 ,    0.      ], dtype=float32)

In [20]:
encoder.save('<location>/model/00000001/') # Put the path to the folder where you want to save the model, don't change last 2 section

INFO:tensorflow:Assets written to: E:/allModels/model/00000001/assets


INFO:tensorflow:Assets written to: E:/allModels/model/00000001/assets


In [21]:
import tarfile
import os

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile("<location>/model.tar.gz", "<location>/model/")

In [ ]:
# from tensorflow.keras.models import model_from_json

# # serialize model to json
# json_model = encoder.to_json()

# # save the model architecture to JSON file
# with open('model.json', 'w') as json_file:
#     json_file.write(json_model)

# # save the trained weights
# encoder.save_weights('model.h5')